<a href="https://colab.research.google.com/github/uneiz1103/Birthday_wish/blob/main/live_api_data_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import numpy as np
import pandas as pd

! pip install requests

In [56]:
import requests

In [57]:
# importing the countries
url = "https://api.worldbank.org/countries?format=json&per_page=300"

response = requests.get(url)
response.status_code


200

In [58]:
data = response.json()
print(data[0])
print(len(data))

{'page': 1, 'pages': 1, 'per_page': '300', 'total': 296}
2


In [59]:
countries = data[1]
countries = pd.DataFrame(countries)

In [60]:
countries["incomeLevel"][0]

{'id': 'HIC', 'iso2code': 'XD', 'value': 'High income'}

In [61]:
countries["region"] = countries["region"].apply(lambda x:x['value'])

In [62]:
countries["incomeLevel"] = countries["incomeLevel"].apply(lambda x:x['value'])

In [63]:
countries["lendingType"] = countries["lendingType"].apply(lambda x:x['value'])

In [64]:
countries.drop(columns=["adminregion", "adminregion", "lendingType"], inplace=True)

In [65]:
countries.drop(columns=["capitalCity"], inplace=True)

In [66]:
countries['region'].unique()

array(['Latin America & Caribbean ', 'Aggregates',
       'Middle East, North Africa, Afghanistan & Pakistan',
       'Sub-Saharan Africa ', 'Europe & Central Asia',
       'East Asia & Pacific', 'South Asia', 'North America'], dtype=object)

In [67]:
countries[countries["region"] == "Aggregates"]

,id,iso2Code,name,region,incomeLevel,longitude,latitude
1,AFE,ZH,Africa Eastern and Southern,Aggregates,Aggregates,,
3,AFR,A9,Africa,Aggregates,Aggregates,,
4,AFW,ZI,Africa Western and Central,Aggregates,Aggregates,,
8,ARB,1A,Arab World,Aggregates,Aggregates,,
18,BEA,B4,East Asia & Pacific (IBRD-only countries),Aggregates,Aggregates,,
...,...,...,...,...,...,...,...
269,TSA,T5,South Asia (IDA & IBRD),Aggregates,Aggregates,,
270,TSS,T6,Sub-Saharan Africa (IDA & IBRD countries),Aggregates,Aggregates,,
278,UMC,XT,Upper middle income,Aggregates,Aggregates,,
288,WLD,1W,World,Aggregates,Aggregates,,


In [71]:
# code for indicators
base_url = 'https://api.worldbank.org/v2/indicators?format=json'
response = requests.get(base_url)


<Response [200]>

In [73]:
indicators_data = response.json()

In [74]:
indicators_data[0]

{'page': 1, 'pages': 1, 'per_page': '300', 'total': 296}

In [76]:
response.json()

[{'message': [{'id': '120',
    'key': 'Invalid value',
    'value': 'The provided parameter value is not valid'}]}]

In [98]:
all_dfs = []
for i in range(1,526):
  url = f"https://api.worldbank.org/v2/indicators?format=json&per_page=500&page={i}"
  response= requests.get(url)

  if response.status_code == 200:
    data = response.json()

    if len(data) < 2:
      print(f"No data found for page {i}")

    indicators = data[1]
    df =pd.DataFrame([{"id": item["id"],
                  "name": item["name"]} for item in indicators])

    all_dfs.append(df)
    print(f"Page{i}: {len(df)} indicators collected")
  else:
    print(f"Failed to retrieve data for page {i}. Status code: {response.status_code}")


Page1: 500 indicators collected
Page2: 500 indicators collected
Page3: 500 indicators collected
Page4: 500 indicators collected
Page5: 500 indicators collected
Page6: 500 indicators collected
Page7: 500 indicators collected
Page8: 500 indicators collected
Page9: 500 indicators collected
Page10: 500 indicators collected
Page11: 500 indicators collected
Page12: 500 indicators collected
Page13: 500 indicators collected
Page14: 500 indicators collected
Page15: 500 indicators collected
Page16: 500 indicators collected
Page17: 500 indicators collected
Page18: 500 indicators collected
Page19: 500 indicators collected
Page20: 500 indicators collected
Page21: 500 indicators collected
Page22: 500 indicators collected
Page23: 500 indicators collected
Page24: 500 indicators collected
Page25: 500 indicators collected
Page26: 500 indicators collected
Page27: 500 indicators collected
Page28: 500 indicators collected
Page29: 500 indicators collected
Page30: 500 indicators collected
Page31: 500 indicat

In [92]:
final_df = pd.concat(all_dfs, ignore_index=True)

In [97]:
fd = final_df.to_csv("final_df.csv")

In [95]:
indicator_groups = {
"economic_activity_growth": [
"NY.GDP.MKTP.KD.ZG", # GDP growth (annual %)
"NY.GDP.PCAP.CD" # GDP per capita (current US$)
],
"labour_market_indicators": [
"SL.UEM.TOTL.ZS", # Unemployment total
"SL.UEM.1524.ZS", # Unemployment youth total (ages 15–24)
"SL.TLF.TOTL.IN" # Labour force, total
],
"trade_globalization": [
"NE.EXP.GNFS.CD", # Exports of goods and services (current US$)
"NE.IMP.GNFS.CD" # Imports of goods and services (current US$)
],
"poverty_inequality": [
"SI.POV.NAHC", # Poverty headcount ratio at national poverty lines (% of population)
"SI.POV.GINI" # Gini index (measure of income inequality)
],
"environmental_indicators": [
"EG.FEC.RNEW.ZS", # Renewable energy consumption (% of total final energy consumption)
"AG.LND.FRST.ZS" # Forest area (% of land area)
],
"health_indicators": [
"SP.DYN.LE00.IN", # Life expectancy at birth
"SP.DYN.IMRT.IN", # Infant mortality rate
"SH.H2O.BASW.ZS", # Access to at least basic water services (% of population)
"SH.XPD.CHEX.GD.ZS", # Current health expenditure (% of GDP)
"SH.IMM.IDPT", # Immunization, DPT (% of children ages 12–23 months)
"SH.IMM.MEAS", # Immunization, measles (% of children ages 12–23 months)
"SH.MMR.RISK.ZS", # Risk of maternal death
"SH.DTH.COMM.ZS", # Deaths from communicable diseases (% of total)
"SH.TBS.INCD", # Tuberculosis incidence (per 100,000 people)
"SH.STA.BRTC.ZS", # Births attended by skilled health staff (%)
"SH.STA.MMRT", # Maternal mortality ratio (modeled estimate, per 100,000 live births)
"SP.POP.65UP.TO.ZS", # Population ages 65 and above (% of total population)
"SH.HIV.INCD.ZS" # HIV incidence rate (per 1,000 uninfected population ages 15–49)
],
"technology_indicators": [
"IT.NET.USER.ZS", # Individuals using the Internet (% of population)
"IT.CEL.SETS.P2" # Mobile cellular subscriptions (per 100 people)
]}